In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

!pip install --upgrade implicit

In [ ]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

In [ ]:
%%time

base_path = '../input/h-and-m-personalized-fashion-recommendations/'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(csv_sub)
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

In [ ]:
df['t_dat']

In [ ]:
# Trying with less data(少ないデータで試す):
# https://www.kaggle.com/tomooinubushi/folk-of-time-is-our-best-friend/notebook
df = df[df['t_dat'] > '2020-08-21']
df.shape

In [ ]:
# For validation this means 3 weeks of training and 1 week for validation
  #バリデーションの場合は、トレーニングに3週間、バリデーションに1週間ということになります。
# For submission, it means 4 weeks of training
  #投稿の場合は、4週間のトレーニングということになります
df['t_dat'].max()

In [ ]:
# Assign autoincrementing ids starting from 0 to both users and items
# ユーザーとアイテムの両方に0から始まる自動インクリメントのidを割り当てる。

#ンクリメントとは、増加、増分などの意味の英単語だが、コンピュータでは数値に1を加える操作のことを指す

In [ ]:
df.head()

In [ ]:
ALL_USERS = dfu['customer_id'].unique().tolist()
ALL_ITEMS = dfi['article_id'].unique().tolist()

#上では、それぞれのカスタマー名と商品名の固有の値を抽出させ、リストに変換させている

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

# enumerate()を用いることにより、インデックス番号, 要素と取得することが出来る。
 #商品、アイテム名を番号処理することが可能。

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

df['user_id'] = df['customer_id'].map(user_map)
df['item_id'] = df['article_id'].map(item_map)

del dfu, dfi

In [ ]:
df.head()

##これで、ユーザーIDとアイテムIDを扱い安やすくすることができた。

In [ ]:
#Create coo_matrix (user x item) and csr matrix (user x item)
#coo_matrix (user x item) と csr matrix (user x item) を作成する。

In [ ]:
row = df['user_id'].values
col = df['item_id'].values

print(row)
print(col)
data = np.ones(df.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

In [ ]:
validation_cut = df['t_dat'].max() - pd.Timedelta(7,"d")
print(validation_cut)

In [ ]:
pd.Timedelta(7, "d")

In [ ]:
print(df['t_dat'].max())
print(validation_cut)

In [ ]:
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(coo_train)

In [ ]:
df.head()

In [ ]:
row  = np.array([0,3,1,0]) # user_ids
col  = np.array([0,3,1,2]) # item_ids
data = np.array([4,5,7,9]) # a bunch of ones of lenght unique(user) x unique(items)
coo_matrix((data,(row,col)), shape=(4,4)).todense()

In [ ]:
coo_matrix((data,(row,col)), shape=(4,4))

In [ ]:
def to_user_item_coo(df):
    "Turn a dataframe with transactions into a COO sparse items x users matrix"
    " トランザクションを含むデータフレームをCOOスパースアイテム×ユーザー行列に変換する"
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo


def split_data(df, validation_days=7):
    """ Split a pandas dataframe into training and validation data, using <<validation_days>>
    pandas のデータフレームを <<validation_days で学習データと検証データに分割する
    """
    validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days,"d")
    df_train = df[df['t_dat'] < validation_cut]
    df_val = df[df['t_dat'] >= validation_cut]
    return df_train, df_val
    

def get_val_matrices(df, validation_days=7):
    """ Split into training and validation and create various matrices
    トレーニング用と検証用に分割し、各種マトリクスを作成する
        
         Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
             csr_train: training data in CSR sparse format and as (users x items)
             csr_val:  validation data in CSR sparse format and as (users x items)
    """
    
    df_train, df_val = split_data(df, validation_days=validation_days)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension) 
    for <<iterations>> over matrices and validate with MAP@12
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    # factors:    計算する潜在的な要因の数
    # iterations: データをフィットする際に使用する ALS の反復回数
    # regularization: 使用する正則化係数(L1正則化)
    
    model.fit(coo_train, show_progress=show_progress)
    #show_progress 学習の進捗を確認
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [ ]:
print(model)

In [ ]:
matrices = get_val_matrices(df)

In [ ]:
%%time
best_map12 = 0
for factors in [10, 25, 50, 75, 100]:
    for iterations in [3, 12, 14, 15, 20]:
        for regularization in [0.01]:
            map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"Best MAP@12 found. Updating: {best_params}")

In [ ]:
del matrices

In [ ]:
#Training over the full dataset

In [ ]:
coo_train = to_user_item_coo(df)
csr_train = coo_train.tocsr()

In [ ]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

In [ ]:
best_params

In [ ]:
model = train(coo_train, **best_params)

In [ ]:
def submit(model, csr_train, submission_name="submissions.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [ ]:
%%time
df_preds = submit(model, csr_train);